In [ ]:
#!/usr/bin/env python
# ------------------------------------------------------------------------------------------------------%
# Created by "Chia E Tungom" 09/15/2020                                                                 %
#       Email:      chemago99@yahoo.com                                                                 %
#       Github:     https://github.com/tungom                                                           %
#-------------------------------------------------------------------------------------------------------%

In [1]:
import numpy as np
import pandas as pd
import copy
import random

# Define Problem Information Functions

In [2]:
def Generate_company_budget(NC, BR = (200, 2000)):
    """ NC: number of companies an interger
        BR: Budget range and is a tuple (lower bound, upper bound)
    returns a list the size of number of companies with 
    randomly assigned budgets"""
    
    return np.random.randint(BR[0],BR[1], NC)

def Generate_price_per_click(NC, NQ, PR = (1, 5)):
    """ PR: the price per click range. Has to be lower than company budget
    
    returns an NC X NQ array"""
    
    return np.random.randint(PR[0],PR[1], (NC,NQ))

def Generate_CTR(NC, NQ):
    """ returns an NC by NQ array with probabilities"""
    
    return np.random.random((NC, NQ))

def Get_APD(PPC, CTR):
    """ multiplies PPC and CTR element wise
    returns an array same size as PPC and CTR """
    
    return np.multiply(PPC, CTR)

def Generate_QE(NQ, ER = (200, 500)):
    """ ER: query estimate range
    retuens a 1D array the lenth of NQ"""
    
    return np.random.randint(ER[0], ER[1], NQ)

# Define Objective Function 

In [3]:
def Initialize_decision_variables(NC, NQ, ND = (1,10)):
    """ return an NC by NQ array with all zeros.
    Zeros becuase no adds are displayed initially
    ND: number of displays (lower_bound, upper_bound)"""
    
    return np.random.randint(ND[0], ND[1], size=(NC, NQ))

def optimization_function(decision_variables, price_per_display, company_budgets, query_estimates):
    # compute budget and queries for each company
    used_budget = np.sum( np.multiply(price_per_display, decision_variables), axis=1)
    used_queries = np.sum(decision_variables, axis = 1)
    
    #print(used_budget)
    #print(used_queries)
    
    voilated_budget_cost = 0
    voilated_query_cost = 0
    voilated_constraints = 0
    
    # budget constraints verification
    for i in range(len(list(used_budget))):
        if used_budget[i] > company_budgets[i]:
            voilated_constraints += 1
            voilated_budget_cost += used_budget[i]
            
    # query constraints verification
    for i in range(len(list(used_queries))):
        if used_queries[i] > query_estimates[i]:
            voilated_constraints += 1
            voilated_query_cost += used_queries[i]
    
    # fitness (total revenue)
    if voilated_constraints > 0:
        punished_fitness = np.sum(np.multiply(decision_variables, price_per_display)) - (voilated_budget_cost + voilated_query_cost)
        fitness = np.sum(np.multiply(decision_variables, price_per_display)) - \
                                    (voilated_budget_cost + voilated_query_cost)**(1+voilated_constraints)
    else:
        punished_fitness = np.sum(np.multiply(decision_variables, price_per_display))
        fitness = punished_fitness
        
    return {"fitness": fitness, "voilated_constraints": voilated_constraints, "punished_fitness": punished_fitness }

# Generate Problem Data

In [4]:
# Define Number of companies NC and number of queries NQ

def data(number_of_companies, number_of_queries, budget_range = (200, 2000), 
         price_range = (1, 20), query_estimate_range = (200, 500) ):
    
    random.seed(100)
    company_budgets = Generate_company_budget(number_of_companies, budget_range)
    price_per_click = Generate_price_per_click(number_of_companies, number_of_queries, price_range)
    click_through_rate = Generate_CTR(number_of_companies, number_of_queries)
    average_price_per_display = Get_APD(price_per_click, click_through_rate)
    query_estimate = Generate_QE(number_of_queries, query_estimate_range)
    
    return {"company_budgets":company_budgets, 
            "average_price_per_display":average_price_per_display, 
            "query_estimate":query_estimate  }

In [5]:
NC, NQ = 5, 10

Data = data(NC, NQ)
decision_variables = Initialize_decision_variables(NC, NQ, ND = (0,10))

# Design of Particle Swarm Optimization Algorithm

# 2.1 Define Initialization and Probability Compututation

In [6]:
def initialize(function, population, price_per_display, company_budgets, query_estimates, bounds = (0, 10)):
    """ function: this is the objective function to be optimized
        population: the number of particles we want in the swarm
        dimensions: int
        bounds: a list containing 2 dimensional tuple with upper and lower bound for each dimension e.g [(a,b),...,(y,z)]
                If bounds are not given the default will be (-100, 100) for each dimension """
    
    NC, NQ = len(company_budgets), len(query_estimates)
    #if bounds == (-10, 10):
    #    bound = [bounds for i in range(dimension)]
    
    pos = [Initialize_decision_variables(NC, NQ, bounds) for j in range(population)]
    vel = [Initialize_decision_variables(NC, NQ, bounds) for j in range(population)]
    
    pos_cost = [function(i, price_per_display, company_budgets, query_estimates) for i in pos]
    
    swarm = {}
    for i in range(population):
        swarm["particle" + str(i)] = {"velocity": vel[i],
                                      "position": pos[i], 
                                      "pbest": pos[i], 
                                      "position_cost": pos_cost[i]["fitness"],
                                      "punished_position_cost": pos_cost[i]["punished_fitness"],
                                      "pbest_cost": pos_cost[i]["fitness"],
                                      "punished_pbest_cost": pos_cost[i]["punished_fitness"],
                                      "position_voilated_constraints": pos_cost[i]["voilated_constraints"],
                                      "pbest_voilated_constraints": pos_cost[i]["voilated_constraints"]}
    return swarm, swarm['particle0']

def probability(swarm):
    """ Calculate the probability of each particle in the swarm """
    #print([i for i in swarm])
    
    denominator = sum([1/( 1 + swarm[particle]['pbest_cost']) for particle in swarm])
    for particle in swarm:
        numerator = 1/( 1 + swarm[particle]['pbest_cost'])
        swarm[particle]['probability'] = numerator/denominator
    
    n = len(swarm)
    for j in range(n):
        working_particle = 'particle' + str(j)
        swarm[working_particle]['roulette_probability'] = sum([swarm['particle' + str(i)]['probability'] for i in range(j+1)])
        
    return swarm

# 2.2 Define Solution Update Approach

In [7]:
# Learning Agent same at every dimension

def position_update(function, swarm, gbest, price_per_display, company_budgets, query_estimates,
                     bounds = (0, 10), w = 0.5, c1 = 1.5, c2 = 1.5):
    
    # get size of position array
    size = swarm['particle0']['pbest'].shape
    
    #if bounds == (-10, 10):
        #bounds = [bounds for i in range(len(swarm['particle0']['position']))]
    
    for particle in swarm:
        
        #SELECT A LEARNING AGENTS -------------------------------------------------------------------------------
        ra = random.random()
        for k in range(len(swarm)):
            if k == 0:
                if ra > 0 and ra < swarm['particle' + str(k)]['roulette_probability']:
                    agent1 = swarm['particle' + str(k)]
                    break
            else:
                if ra > swarm['particle' + str(k-1)]['roulette_probability'] and ra < swarm['particle' + str(k)]['roulette_probability']:
                    agent1 = swarm['particle' + str(k)]
                    #print("selelcted an agent ", agent1)
                    break
                
        ra = random.random()
        for k in range(len(swarm)):
            if k == 0:
                if ra > 0 and ra < swarm['particle' + str(k)]['roulette_probability']:
                    agent2 = swarm['particle' + str(k)]
                    break
            else:
                if ra > swarm['particle' + str(k-1)]['roulette_probability'] and ra < swarm['particle' + str(k)]['roulette_probability']:
                    agent2 = swarm['particle' + str(k)]
                    #print("selelcted an agent ", agent2)
                    break
       
        # UPDATE VELOCITY --------------------------------------------------------------------------------------
        r1, r2 = np.random.uniform(0,1, size), np.random.uniform(0,1, size)
            
        position = w*swarm[particle]['position'] +\
                   c1*r1*(swarm[particle]['pbest'] - swarm[particle]['position']) +\
                   c2*r2*(gbest['pbest'] - swarm[particle]['position'])
        
        # curb position into bounds
        # https://moonbooks.org/Articles/How-to-replace-some-elements-of-a-matrix-using-numpy-in-python-/
        position = np.round(position, 0)
        position[(position < bounds[0])] = random.randint(bounds[0],bounds[1])
        position[(position > bounds[1])] = random.randint(bounds[0],bounds[1])
        
        # add updated velocity to particle information  and updated position to the position 
        #swarm[particle]["velocity"] = updated_velocity
        swarm[particle]["position"] = position
        updates = function(position, price_per_display, company_budgets, query_estimates)
        swarm[particle]["position_cost"] = updates["fitness"]
        swarm[particle]["punished_position_cost"] = updates["punished_fitness"]
        swarm[particle]["position_voilated_constraints"] = updates["voilated_constraints"]
        
        # UPDATE PBEST AND GBEST-------------------------------------------------------------------------------------
        # print(swarm[particle]["position_cost"])
        if swarm[particle]["position_cost"] > swarm[particle]["pbest_cost"]:
            swarm[particle]["pbest"] = position
            swarm[particle]["pbest_cost"] = swarm[particle]["position_cost"]
            swarm[particle]["punished_pbest_cost"] = swarm[particle]["punished_position_cost"]
            swarm[particle]["pbest_voilated_constraints"] = swarm[particle]["position_voilated_constraints"]
            
        
        # print("GBEST  ", gbest["pbest_cost"])
        # print("PBEST  ", swarm[particle]["pbest_cost"])
        if swarm[particle]["pbest_cost"] > gbest["pbest_cost"]:
            gbest = swarm[particle]
            #print("CHange in Gbest")
            
    return swarm, gbest

# 2.3 Define The PSO optimization Algorithm 

In [8]:
def PSO(function, population, price_per_display, company_budgets, query_estimates,
        generations = 500, w = 0.6, c1 = 1.7, c2 = 0.7, bounds = (0, 10)):
    
    print("population ", population, " generarions ", generations)
    
    Swarm, gbest = initialize(function, population, price_per_display, company_budgets, query_estimates, bounds)
    Evolutionary_Swarm = [copy.deepcopy(Swarm)]
    #w, c1, c2 = 0.6, 1.5, 0.5
    
    i = 0
    while generations > i :
        
        probability(Swarm)
        Swarm, gbest = position_update(function, Swarm, gbest, price_per_display, company_budgets, query_estimates,
                     bounds, w = 0.5, c1 = 1.5, c2 = 1.5)
        Evolutionary_Swarm.append(copy.deepcopy(Swarm))
        i += 1
        
        if i%int(generations*0.01) == 0 :
            print(" Generation ", i , "best cost ", gbest['pbest_cost'], "constraints voilated", gbest['pbest_voilated_constraints'])
        
    return {"evolved_swarm":Evolutionary_Swarm, "best_particle": gbest}

# 3.1 Define the Problem and Solve with the designed Algorithm

In [9]:
# problem parameters
Number_of_companies, Number_of_queries = 5, 5
Data = data(Number_of_companies, Number_of_queries)
#decision_variables = Initialize_decision_variables(Number_of_companies, Number_of_queries, ND = (1,10))
price_per_display = Data["average_price_per_display"] 
company_budgets = Data["company_budgets"] 
query_estimates = Data["query_estimate"]

# Algorithm paramters
population = 1
generations = 100
w, c1, c2 = 0.6, 2.0, 2.0


price_per_display.shape
len(company_budgets)
len(query_estimates)

5

In [10]:
EvSwarm = PSO(optimization_function, population, price_per_display, company_budgets, query_estimates,
              generations, w , c1 , c2 )

population  1  generarions  100
 Generation  1 best cost  665.8836916251397 constraints voilated 0
 Generation  2 best cost  665.8836916251397 constraints voilated 0
 Generation  3 best cost  665.8836916251397 constraints voilated 0
 Generation  4 best cost  665.8836916251397 constraints voilated 0
 Generation  5 best cost  665.8836916251397 constraints voilated 0
 Generation  6 best cost  665.8836916251397 constraints voilated 0
 Generation  7 best cost  665.8836916251397 constraints voilated 0
 Generation  8 best cost  665.8836916251397 constraints voilated 0
 Generation  9 best cost  665.8836916251397 constraints voilated 0
 Generation  10 best cost  665.8836916251397 constraints voilated 0
 Generation  11 best cost  665.8836916251397 constraints voilated 0
 Generation  12 best cost  665.8836916251397 constraints voilated 0
 Generation  13 best cost  665.8836916251397 constraints voilated 0
 Generation  14 best cost  665.8836916251397 constraints voilated 0
 Generation  15 best cost

In [176]:
EvSwarm["best_particle"]

{'pbest': array([[ 2.,  9.,  9.,  6.,  0.],
        [ 6.,  0.,  7.,  8.,  2.],
        [ 7.,  4.,  2.,  8.,  0.],
        [10., 10.,  3., 10.,  1.],
        [ 5.,  6.,  2.,  2.,  0.]]),
 'pbest_cost': 592.9937471730648,
 'pbest_voilated_constraints': 0,
 'position': array([[2., 9., 7., 3., 0.],
        [2., 0., 3., 9., 1.],
        [5., 3., 1., 4., 0.],
        [9., 7., 5., 6., 0.],
        [3., 2., 2., 2., 0.]]),
 'position_cost': 490.13484210209793,
 'position_voilated_constraints': 0,
 'probability': 1.0,
 'punished_pbest_cost': 592.9937471730648,
 'punished_position_cost': 490.13484210209793,
 'roulette_probability': 1.0,
 'velocity': array([[3, 8, 8, 2, 7],
        [0, 5, 5, 6, 7],
        [6, 7, 7, 8, 2],
        [2, 5, 2, 9, 2],
        [9, 0, 0, 8, 8]])}